In [ ]:
# Libraries

import pandas as pd
import numpy as np
import re
from pprint import pprint
from sklearn.manifold import TSNE
from scipy.cluster.hierarchy import fcluster
from sklearn.preprocessing import normalize
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# Reading CSV files

optim_f_df=pd.read_csv('data/BGEIDSC.EF2EFFP.csv')                   # fuel optimization 
trans_f_df=pd.read_csv('data/BGETCHDATA.PTCHTRANH.csv')              # fuel transactions
event_f_df=pd.read_csv('data/Extranet2.ExactFuelEvents.csv')         # fuel events
level_f_df=pd.read_csv('data/Extranet2.ExactFuelTankLevels.csv')     # fuel levels
truck_v_df=pd.read_csv('data/IBGEFILE.UNITS.csv')                    # trucks 
perfo_v_df=pd.read_csv('data/Extranet2.QCPerformanceExtracts.csv')   # vehicle performance


## Overviewing Data

In [ ]:
event_f_df.head(1)
level_f_df.head()

#optim_f_df.head(3)

# TRUCK     Unit Number               *****                                           [TU]
# ACTIVE    Active Flag (for fuel optimization)
# SENDFUEL  Send Fuel Flag (for fuel optimization)
# SENDROUTE Send Route Flag (for fuel optimization)
# TANKCAP   Tank Capacity (gallons)                                       * 5 types
# AVGMPG    MPG used for Fuel optimization (real value is closer to 7mpg)


#trans_f_df.head(3)

# TRNID   Transaction ID 
# TRNDAT  Transaction Date (YYYYMMDD)
# TRNTIM  Transaction Time (HHMM)
# TRNTS#  Station ID
# TRNTSN  Station Name
# TRNSTC  Station City
# TRNST   Station State
# TRNDRI  Driver Code                                                          [DR]
# TRNUNT  Unit ID                                                              [TU]
# TRNPI1  Item Code 1
# TRNPQ1  Item Quantity 1
# TRNPI2  Item Code 2
# TRNPQ2  Item Quantity 2
# TRNPI3  Item Code 3
# TRNPQ3  Item Quantity 3
# TRNPI4  Item Code 4
# TRNPQ4  Item Quantity 4
# TRNPI5  Item Code 5
# TRNPQ5  Item Quantity 5
# TRNPI6  Item Code 6
# TRNPQ6  Item Quantity 6

# Actual fuel transactions will have an item code of ULSD or FUEL, CDSL, DSL1, BDSL, 
# and could be in any of the Item slots 1-6

level_f_df.head(3)

# Id                AutoNumber id
# TankId            Tank Sensor #         (for our trucks, will always be 1)
# TankLevelPercent  Percentage Reading    0-100.00
# TankLevelGallons  Gallons               (Percentage * Tank Capacity) (calculated)
# ExactFuelEventId  Foreign Key to ExactFuelEvent                                       [EV]

In [ ]:
# This block creates a list of truck IDs as numeric values (valid_trucks).

good_trux_3 = optim_f_df['TRUCK']
list_of_trucks_3 = list(set(good_trux_3))
valid_trucks_o = [x for x in list_of_trucks_3 if (x >= 1 | x < 9999)]
len(valid_trucks_o)
#418

In [ ]:
# This block creates a list of truck IDs as numeric values (valid_trucks).

#good_trux = trans_f_df['TRNUNT'].str.extract('(\d+)')
#list_of_trucks = list(set(good_trux))
#num_trux = pd.to_numeric(list_of_trucks, downcast='integer')
#valid_trucks_t = [x for x in num_trux if np.isnan(x) != True]
#print(valid_trucks_t)

good_trux_2 = event_f_df['EquipmentID']
list_of_trucks_2 = list(set(good_trux_2))
valid_trucks_e = [x for x in list_of_trucks_2 if (x >= 1 | x < 9999)]
len(valid_trucks_e)
#327

In [ ]:
# This block creates a list of driver codes.

good_driv = trans_f_df['TRNDRI'].str.extract('(\S+)')
list_of_drivs = list(set(good_driv))
valid_driv_t = [x for x in list_of_drivs if str(x) != 'nan']

len(valid_driv_t)
#938

In [ ]:
# transaction codes for fuel transactions
trx_list = ['ULSD', 'FUEL', 'CDSL', 'DEFC', 'DSL1', 'BDSL']
df_fuel_trans = trans_f_df[trans_f_df['TRNPI1'].isin(trx_list)]
# We don't need the TRNPI 2-6 column since they do not contain any valid fuel item (I have checked them 
# but the queries are not here.)
df_fuel_trans = df_fuel_trans.drop(['TRNPI2','TRNPI3','TRNPI4','TRNPI5','TRNPI6','TRNPQ2','TRNPQ3',\
                                    'TRNPQ4','TRNPQ5','TRNPQ6'], axis=1)
df_fuel_trans = df_fuel_trans.drop(['TRNID','TRNDAT','TRNTIM','TRNTS#','TRNTSN','TRNSTC','TRNST', 'TRNPI1'], axis=1)
df_fuel_trans.head(5)
df_fuel_trans.shape
# (104057, 3)
#df_fuel_trans = df_fuel_trans[df_fuel_trans.TRNDRI.isin(valid_driv_t)]
#df_fuel_trans.shape
#df_fuel_trans = df_fuel_trans[df_fuel_trans.TRNUNT.isin(valid_trucks_e)]
#df_fuel_trans.shape
#df_fuel_trans = df_fuel_trans[(df_fuel_trans.TRNDRI.isin(valid_driv_t) == True) & (df_fuel_trans.TRNUNT.isin(valid_trucks_e) == True)]


#df_fuel_trans.head(5)


In [ ]:
# this converts blanks to string and then strip will make it a Nan and then it can be removed
df_fuel_trans['TRNUNT'] = df_fuel_trans['TRNUNT'].apply(str)
df_fuel_trans['TRNUNT'] = df_fuel_trans['TRNUNT'].str.strip()
df_fuel_trans['TRNDRI'] = df_fuel_trans['TRNDRI'].apply(str)
df_fuel_trans['TRNDRI'] = df_fuel_trans['TRNDRI'].str.strip()
df_fuel_trans.isnull().values.any()
print(df_fuel_trans['TRNUNT'].dtype)
print(df_fuel_trans['TRNDRI'].dtype)

#This gets us a compact df with driver, truck and fuel level across all fuel transactions

In [ ]:


df_fuel_by_truck_driver = df_fuel_trans.groupby(['TRNDRI','TRNUNT'], as_index=False)['TRNPQ1'].sum()
#df_fuel_by_truck_driver['TRNDRI'].replace('', np.nan, inplace=True)   
#df_fuel_by_truck_driver['TRNUNT'].replace('', np.nan, inplace=True)
#df_fuel_by_truck_driver.dropna(axis=0, how='any')
#df_fuel_by_truck_driver = df_fuel_by_truck_driver[~(df_fuel_by_truck_driver[['TRNDRI']] == '').any(axis=1)]
#df_fuel_by_truck_driver = df_fuel_by_truck_driver[df_fuel_by_truck_driver['TRNDRI'].notnull() & df_fuel_by_truck_driver['TRNUNT'].notnull()]
df_fuel_by_truck_driver = df_fuel_by_truck_driver.rename(columns={'TRNDRI':'DriverID', 'TRNUNT':'EquipmentID', \
                                                                  'TRNPQ1':'fuel'})
df_fuel_by_truck_driver.head()
#df_fuel_by_truck_driver = df_fuel_by_truck_driver.reset_index()

#df_fuel_by_truck_driver = df_fuel_by_truck_driver.reset_index()
#df_fuel_by_truck_driver.info()
df_fuel_by_truck_driver.head()
#df.groupby(['Country', 'Item_Code'])[["Y1961", "Y1962", "Y1963"]].sum()


In [ ]:
event_f_df.head(3)

# Id                AutoNumber ID                                                   [EV]
# ESS_Id            Omnitracs Event Subscriber Service Event Id
# EventTimeStamp    DateTime of Event (normalized to Central Time)
# EquipmentID       Unit Number                                                     [TU] (?)
# MCTNumber         Mobile Communications Terminal ID
# EquipmentType     should always read ‘tractor’
# DriverId          Driver Code Assigned to unit at time of reading (varchar(6))    [DR] (?)
# Latitude          decimal degrees latitude at time of reading
# Longitude         decimal degrees longitude at time of reading
# LocationTimeStamp DateTime of location reading
# Speed             MPH at time of reading
# Heading           direction of travel at time of reading
# Odometer          unit odometer at time of reading
# IgnitionStatus    1=on, 2=off
# EFReportReason    0=ignition on, 1=ignition off, 2=timer

# NOTE: Discard equipment IDs that are wrongfully replaced by MCTNumber
event_f_df.shape
#(8495130, 15)

In [ ]:
# how many truck events for each truck??
df_events_trucks = event_f_df[~(event_f_df[['Odometer']] == 0).any(axis=1)]

#df_events_truck = df_events_truck.EquipmentID.value_counts().sort_values(ascending=False)
#df_events_truck = pd.DataFrame(data=df_events_truck)
#df_events_truck = df_events_truck.reset_index();
#df_events_truck.columns = ['EquipmentID','Count']
#df_events_truck.head()
df_events_trucks.head()
df_events_trucks.shape
#df_events_trucks.info()

In [ ]:
# Calculate the miles travelled by driver truck combination
df_events_trucks['DriverID'] = df_events_trucks['DriverID'].apply(str)
df_events_trucks['EquipmentID'] = df_events_trucks['EquipmentID'].apply(str)
df_events_trucks['DriverID'] = df_events_trucks['DriverID'].str.strip()
df_events_trucks['EquipmentID'] = df_events_trucks['EquipmentID'].str.strip()

df_events_trucks.isnull().values.any()
print(df_events_trucks['DriverID'].dtype)
print(df_events_trucks['EquipmentID'].dtype)

df_mileage = pd.DataFrame()
df_mileage['lowest'] = df_events_trucks.groupby(['DriverID','EquipmentID'])['Odometer'].min()
df_mileage['highest'] = df_events_trucks.groupby(['DriverID','EquipmentID'])['Odometer'].max()
df_mileage['miles'] = (df_mileage['highest'] - df_mileage['lowest'])

df_mileage.dropna()
df_mileage = df_mileage.reset_index()
df_mileage.head()
#df_mileage.info()
#filtered_data = df[df.topping == 'pineapple']
great_driver_list = ['SMID', 'BEVL', 'RICS', 'NEWR', 'MARC', 'LOVH']
#df_mileage[df_mileage.DriverID.isin(great_driver_list)]
#df_mpg.loc[df_mpg['DriverID'] =='LOVH',:]  
#filtered_data

In [ ]:
#df_mileage[df_mileage.DriverID.isin(great_driver_list)]

In [ ]:
#df_mileage.head()
#df_mileage.info()

In [ ]:
#df_fuel_by_truck_driver.head()
#df_fuel_by_truck_driver.info()


In [ ]:
#df_mileage[df_mileage.DriverID.isin(great_driver_list)]


df_fuel_by_truck_driver['DriverID'] = df_fuel_by_truck_driver['DriverID'].apply(str)
df_mileage['DriverID'] = df_mileage['DriverID'].apply(str)
df_fuel_by_truck_driver['EquipmentID'] = df_fuel_by_truck_driver['EquipmentID'].apply(str)
df_mileage['EquipmentID'] = df_mileage['EquipmentID'].apply(str)



#df_merged = pd.merge(df_mileage, df_fuel_by_truck_driver,  on=['DriverID','EquipmentID'], how='inner')
df_merged = pd.merge(df_mileage, df_fuel_by_truck_driver, left_on=['DriverID','EquipmentID'], right_on=['DriverID','EquipmentID'], how='inner')
#print(pd.merge(df1,df2, on=['HPI','Int_rate']))

#df_merged[df_merged.DriverID.isin(great_driver_list)]

df_merged.head()
#df_merged.info()
#df_merged.shape
#(2981, 7)
#df_merged.isnull().sum()

In [ ]:
df_merged['mpg'] = df_merged['miles']/df_merged['fuel']
df_mpg = df_merged[(df_merged['mpg'] > 3) & (df_merged['mpg'] < 10)]
df_mpg.head()
#df_merged.shape
#df_mpg.describe()
#df_merged.DriverID.value_counts(dropna = False)

In [ ]:
num_bins = 20
n, bins, patches = plt.hist(df_mpg.mpg, num_bins, facecolor='blue', alpha=0.5)
plt.show()

In [ ]:
# Make a classification based on mpg. Good > 7, Avergage 5.5-7 and Bad < 5.5
df_mpg['class'] = np.where(df_mpg['mpg']>7, 'Good', 'Avergage')
#df_mpg['class'] = np.where((df_mpg['mpg'] >= 5.5) & (df_mpg['mpg'] < 7), 'Average', 'Bad')
df_mpg['class'] = np.where((df_mpg['mpg'] < 5.5) , 'Bad', 'Average')
#df['elderly'] = np.where(df['age']>=50, 'yes', 'no')

In [ ]:
df_mpg.head()

In [ ]:
def classify_driver(row):
    if row['mpg'] >= 7:
        val = 'Good'
    elif row['mpg'] >= 5.5:
        val = 'Average'
    else:
        val = 'Bad'
    return val

df_mpg['class'] = df_mpg.apply(classify_driver, axis=1)
df_mpg.head()

In [ ]:
#num_bins = 3
#n, bins, patches = plt.hist(df_mpg.class, num_bins, facecolor='blue', alpha=0.5)
#plt.show()
df_mpg['class'].value_counts().plot(kind='bar')
plt.show()

In [ ]:
df_mpg.head()

In [ ]:
#plot data with seaborn
facet = sns.lmplot(data=df_mpg, x='EquipmentID', y='mpg', hue='class', 
                   fit_reg=False, legend=True, legend_out=True)
plt.show()

In [ ]:
bplot = sns.boxplot(y='mpg', x='class', data = df_mpg, width=0.8, palette="colorblind")
plt.show()

In [ ]:
#df.loc[df['column_name'] == some_value]
#print(df_mpg.loc[df_mpg['DriverID']=='BEVL'])
df_mpg[df_mpg.DriverID =='MARC']
#df_mpg.head()

In [ ]:
# Add Great Drivers list for comparison and setting values of class
driver_list = ['SMID', 'BEVL', 'RICS', 'NEWR', 'MARC', 'LOVH']
#filtered_data = df_mileage[df_mileage.DriverID == 'MARC']
#driver_list = {'SMID':'Great', 'BEVL':'Great', 'RICS':'Great', 'NEWR':'Great', 'MARC':'Great', 'LOVH':'Great'}
#df_mpg['class'] = np.where[df_mpg['DriverID'].isin(driver_list), 'Great', df_mpg['class']]
#np.where(df_mpg['mpg']>7, 'Good', 'Avergage')
#df_mpg.head()
#d = {'NONANE':9, 'OCTANE':8, 'HEPTANE':7, 'HEXANE':6}
#df['num'] = df['solvent'].map(d)
#df_mpg.loc[df_mpg['DriverID'].isin(driver_list), 'Class'] = 'Great'
df_mpg.loc[df_mpg.DriverID == 'SMID', 'class'] = 'Great'
df_mpg.loc[df_mpg['DriverID'] == 'BEVL', 'class'] = 'Great'
df_mpg.loc[df_mpg['DriverID'] == 'RICS', 'class'] = 'Great'
df_mpg.loc[df_mpg['DriverID'] == 'NEWR', 'class'] = 'Great'
df_mpg.loc[df_mpg['DriverID'] == 'MARC', 'class'] = 'Great'
df_mpg.loc[df_mpg['DriverID'] == 'LOVH', 'class'] = 'Great'
#df_mpg.loc[df_mpg['Class'] == 'Great']
df_mpg.describe()
#df_mpg.loc['Class'] = df_mpg['DriverID'].map(driver_list)
#df_mpg.head(100)
#df_mpg.loc[df_mpg['DriverID'].isin(driver_list), :]
df_mpg.head()
df_mpg[df_mpg.DriverID == 'MARC']

In [ ]:

sns.set(color_codes=True)
sns.distplot(df_mpg.mpg);
plt.show()

In [ ]:
df_mpg['class'].value_counts().plot(kind='bar')
plt.show()

In [ ]:
# Draw a categorical scatterplot to show each observation
sns.set(style="whitegrid", palette="muted")
sns.swarmplot(x="class", y="mpg", hue="class", data=df_mpg)
plt.show()

# Detecting Fraud

In [ ]:
# find bad mpg drivers
df_bad_drivers = df_mpg[df_mpg['class'] == 'Bad']
#filtered_data = df[df.topping == 'pineapple']
df_bad_drivers.head()

In [ ]:
df = pd.merge(event_f_df, level_f_df, how='inner', left_on=['Id'], right_on = ['ExactFuelEventId'])
dfs = df[['EventTimeStamp', 'EquipmentID', 'DriverID', 'Odometer','IgnitionStatus','TankLevelGallons']]
# calculate fuel difference between each set of rows
dfs = dfs[(dfs['DriverID'] == 'BUNC') & (dfs['IgnitionStatus'] == 2)]

dfs = dfs.sort_values(by=['EventTimeStamp','EquipmentID','DriverID','Odometer'])

#dfs['odo_diff'] = dfs['Odometer'].shift(-1) -  dfs['Odometer']
dfs['odo_diff'] = dfs['Odometer'].diff()
dfs['fuel_diff'] = dfs['TankLevelGallons'].diff()
dfs.loc[dfs.EquipmentID != dfs.EquipmentID.shift(), 'odo_diff'] = 0
dfs.loc[dfs.EquipmentID != dfs.EquipmentID.shift(), 'fuel_diff'] = 0


dfs.head()

In [ ]:
df_doubt = dfs[(dfs['odo_diff'] == 0) & (dfs['fuel_diff'] < -1)]
df_doubt

In [ ]:
dq = dfs[(dfs['Odometer'] == 203784.3) ]
dq

In [ ]:
# use the function regplot to make a scatterplot
sns.regplot(x=dq["Odometer"], y=dq["fuel_diff"])
plt.show()
 
# Without regression fit:
#sns.regplot(x=dfs["Odometer"], y=dfs["TankLevelGallons"], fit_reg=False)
#sns.plt.show()


##### Downsizing the databases
To do this, we eliminate all drivers/trucks that have invalid syntax, followed by reducing the databases that contain these elements

In [ ]:
# Here all the valid drivers and trucks are inner-joined (intersected), to be used for downsizing the database.

valid_driv = [x for x in valid_driv_t if (x in valid_driv_e)]

# 2 step inner join for tricks
valid_truck = [x for x in valid_trucks_e if (x in valid_trucks_o)]
valid_trux = [x for x in valid_truck if (x in valid_trucks_t)]

In [ ]:
# Downsizing dataframes take place here.

# optimization
optim_f_df_ds = optim_f_df[optim_f_df.TRUCK.isin(valid_trux) == True]
# events
event_f_df_ds = event_f_df[(event_f_df.EquipmentID.isin(valid_trux) == True) & (event_f_df.DriverID.isin(valid_driv) == True)]
# Transactions
trans_f_df['numtruc'] = pd.to_numeric(trans_f_df['TRNUNT'], errors='coerce')
trans_f_df_ds = trans_f_df[(trans_f_df.TRNDRI.isin(valid_driv) == True) & (trans_f_df.numtruc.isin(valid_trux) == True)]

# Fraudulent Transaction Column

In [ ]:
df_fraud = pd.merge(event_f_df, level_f_df, how='inner', left_on=['Id'], right_on = ['ExactFuelEventId'])
#df_merged = pd.merge(df_mileage, df_fuel_by_truck_driver, on=['DriverID','EquipmentID'])
#new_df = pd.merge(A_df, B_df,  how='left', left_on=['A_c1','c2'], right_on = ['B_c1','c2'])
df_fraud.head()

### Merging data sets and cleaning columns

In [ ]:
# Merge database
fuel_trak_df = pd.merge(event_f_df_ds, level_f_df, how='inner', left_on='Id', right_on='ExactFuelEventId')
# discard duplicate columns
fuel_trak_df = fuel_trak_df.drop(['Id_x','Id_y'], axis=1)

In [ ]:
# Columns added to allocate differences in nextcoming steps

fuel_trak_df = fuel_trak_df.assign(dif_Speed="")
fuel_trak_df = fuel_trak_df.assign(dif_ODO="")
fuel_trak_df = fuel_trak_df.assign(dif_tanklevel="")
fuel_trak_df = fuel_trak_df.assign(dif_tankperc="")

In [ ]:
print(len(valid_trux), len(list(set(fuel_trak_df.EquipmentID))))

In [ ]:
level_f_df.head(3)

In [ ]:
level_f_df.head(3)

In [ ]:
fuel_trak_df.sort_values(by = 'EventTimeStamp', axis=0)

# Data Dictionary
 Fuel Analysis Project
 
###### BGETCHDATA.PTCHTRANH

1.	TRNID		Transaction ID 
2.	TRNDAT	    Transaction Date (YYYYMMDD)
3.	TRNTIM		Transaction Time (HHMM)
4.	TRNTS#		Station ID
5.	TRNTSN		Station Name
6.	TRNSTC		Station City
7.	TRNST		Station State
8.	TRNDRI		Driver Code
9.	TRNUNT	    Unit ID
10.	TRNPI1		Item Code 1
11.	TRNPQ1	    Item Quantity 1
12.	TRNPI2		Item Code 2
13.	TRNPQ2	    Item Quantity 2
14.	TRNPI3		Item Code 3
15.	TRNPQ3	    Item Quantity 3
16.	TRNPI4		Item Code 4
17.	TRNPQ4	    Item Quantity 4
18.	TRNPI5		Item Code 5
19.	TRNPQ5	    Item Quantity 5
20.	TRNPI6		Item Code 6
21.	TRNPQ6      Item Quantity 6

Actual fuel transactions will have an item code of ULSD or FUEL, CDSL, DSL1, BDSL, and could be in any of the Item slots 1-6
 

###### IBGEFILE.UNITS

1.	UNUNIT	Unit Number
2.	UNYEAR	Model Year
3.	UNMAKE	Make
4.	UNSER	VIN

###### BGEIDSC.EF2EFFP

1.	TRUCK		Unit Number
2.	ACTIVE		Active Flag (for fuel optimization)
3.	SENDFUEL	Send Fuel Flag (for fuel optimization)
4.	SENDROUTE	Send Route Flag (for fuel optimization)
5.	TANKCAP	    Tank Capacity (gallons)
6.	AVGMPG	    MPG used for Fuel optimization (real value is closer to 7mpg)

###### Extranet2.ExactFuelEvent

1.	Id			    AutoNumber ID
2.	ESS_Id			Omnitracs Event Subscriber Service Event Id
3.	EventTimeStamp	DateTime of Event (normalized to Central Time)
4.	EquipmentID		Unit Number
5.	MCTNumber		Mobile Communications Terminal ID
6.	EquipmentType	should always read ‘tractor’
7.	DriverId		Driver Code Assigned to unit at time of reading (varchar(6))
8.	Latitude		decimal degrees latitude at time of reading
9.	Longitude		decimal degrees longitude at time of reading
10.	LocationTimeStamp 	DateTime of location reading
11.	Speed			MPH at time of reading
12.	Heading		    direction of travel at time of reading
13.	Odometer		unit odometer at time of reading
14.	IgnitionStatus	1=on, 2=off
15.	EFReportReason	0=ignition on, 1=ignition off, 2=timer

###### Extranet2.ExactFuelTankLevel

1.	Id			        AutoNumber id
2.	TankId			    Tank Sensor # (for our trucks, will always be 1)
3.	TankLevelPercent	Percentage Reading 0-100.00
4.	TankLevelGallons	Gallons (Percentage * Tank Capacity) (calculated)
5.	ExactFuelEventId	Foreign Key to ExactFuelEvent

##### Extranet2.QCPerformanceExtracts
See Mary's post on slack 
https://nashvillesoftware.slack.com/files/U71DCPYBX/FAF8Q7JSZ/screen_shot_2018-04-28_at_9.03.29_am.png

# STEPS TO TAKE
+ Identify invalid Trucks/Driver names
+ Convert into valid syntax (numeric etc)
+ Make an intersection of all Trucks & Drivers and create a final list of good IDs for trucks and drivers
+ Extract records with only "good" IDs (downsizing the database)
+ Merge dataframes (subject to discussion)
+ Added extra columns for allocating differences
+ Sort By time

TO DO:
- Write a FOR loop to 
    a. filter the dataframe by each truck
    b. calculate the differences and allocate them in columns (calculate shift)